# LangGraph 201: Building Multi-Agent Workflows (TypeScript)


In this notebook, we're going to walk through setting up a **multi-agent workflow** in LangGraph using TypeScript and LangChain v1. We will start from a simple ReAct-style agent and add additional steps into the workflow, simulating a realistic customer support example, showcasing human-in-the-loop, long term memory, and the LangGraph pre-built library. 

The agent utilizes the [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/), and is able to handle customer inqueries related to invoice and music. 

![Arch](../../images/architecture.png) 

For a deeper dive into LangGraph primitives and learning our framework, check out our [LangChain Academy](https://academy.langchain.com/courses/intro-to-langgraph)!


## Pre-work: Setup


#### Initialize the LLM

We use OpenAI in this example, but feel free to swap ChatOpenAI with other model providers that you prefer.


In [2]:
import { initChatModel } from "langchain";

// Initialize the model
// Make sure to set your OPENAI_API_KEY environment variable
var model = await initChatModel("openai:o3-mini");

console.log("Model initialized!");


Model initialized!


#### Loading sample customer data

The agent utilizes the [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/), which contains sample information on customer information, purchase history, and music catalog.


In [3]:
// Workaround for tslab module resolution with package subpath exports
// TSLab provides 'require' globally, so we can use it directly
var { SqlDatabase } = require("@langchain/classic/sql_db");

import { DataSource } from "typeorm";
import initSqlJs from "sql.js";

// Download and execute the Chinook SQL script from GitHub
var sqlScriptUrl = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql";

console.log("Downloading Chinook SQL script from GitHub...");
var response = await fetch(sqlScriptUrl);
if (!response.ok) {
  throw new Error(`Failed to download SQL script. Status: ${response.status}`);
}
var sqlScript = await response.text();
console.log("SQL script downloaded successfully!");

// Initialize sql.js and create database from SQL script
console.log("Initializing sql.js and executing SQL script...");
var SQL = await initSqlJs();
var sqlJsDb = new SQL.Database();

// Execute the SQL script to create and populate the database
sqlJsDb.exec(sqlScript);
console.log("Database created and populated successfully!");

// Export database to buffer for TypeORM
var dbBuffer = sqlJsDb.export();

// Create TypeORM DataSource with sql.js
var datasource = new DataSource({
  type: "sqljs",
  database: dbBuffer,
  synchronize: false,
});

// Initialize the DataSource
await datasource.initialize();

var db = await SqlDatabase.fromDataSourceParams({
  appDataSource: datasource
});

console.log("✅ Chinook database loaded successfully from SQL script!");


SQL script downloaded successfully!
Initializing sql.js and executing SQL script...
Database created and populated successfully!
✅ Chinook database loaded successfully from SQL script!


#### Setting up short-term and long-term memory

We will initialize a checkpointer for **short-term memory**, maintaining context within a single thread. 

**Long term memory** lets you store and recall information between conversations. We will utilize our long term memory store to store user preferences for personalization.


In [4]:
import { InMemoryStore, MemorySaver } from "@langchain/langgraph";

// Initialize memory stores
// Using var for better cross-cell availability in tslab
var inMemoryStore = new InMemoryStore();
var checkpointer = new MemorySaver();

console.log("Memory stores initialized!");


Memory stores initialized!


## Part 1: Building The Sub-Agents


### 1.1 Building a ReAct Agent from Scratch

Now that we are set up, we are ready to build out our **first subagent**. This is a simple ReAct-style agent that fetches information related to music store catalog, utilizing a set of tools to generate its response. 

![react_1](../../images/music_subagent.png)


#### State

**State can be thought of as the memory of the agent - it's a shared data structure that's passed on between the nodes of your graph**, representing the current snapshot of your application. 

For our customer support agent, our state will track:
1. The customer ID
2. Conversation history
3. Memory from long term memory store
4. Remaining steps (tracks # steps until it hits recursion limit)


In [5]:
import { BaseMessage } from "langchain";
import { Annotation, messagesStateReducer } from "@langchain/langgraph";

// Define Input State
var InputStateAnnotation = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: messagesStateReducer,
  }),
});

// Define overall State
var StateAnnotation = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: messagesStateReducer,
  }),
  customerId: Annotation<number | undefined>({
    reducer: (x, y) => y ?? x,
    default: () => undefined,
  }),
  loadedMemory: Annotation<string>({
    reducer: (x, y) => y ?? x,
    default: () => "",
  }),
  remainingSteps: Annotation<number>({
    reducer: (x, y) => y ?? x,
    default: () => 25,
  }),
});

console.log("State schemas defined!");


State schemas defined!


#### Tools

Let's define a list of **tools** our agent will have access to. Tools are functions that can act as extension of the LLM's capabilities. We will create several tools that interact with the Chinook database regarding music.

We can create tools using the `tool` decorator.


In [6]:
import { tool } from "langchain";
import { z } from "zod";

// Define music catalog tools
var getAlbumsByArtist = tool(
  async ({ artist }: { artist: string }) => {
    var query = `
      SELECT Album.Title, Artist.Name 
      FROM Album 
      JOIN Artist ON Album.ArtistId = Artist.ArtistId 
      WHERE Artist.Name LIKE '%${artist}%';
    `;
    var result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "get_albums_by_artist",
    description: "Get albums by an artist.",
    schema: z.object({
      artist: z.string().describe("The artist name"),
    }),
  }
);

var getTracksByArtist = tool(
  async ({ artist }: { artist: string }) => {
    var query = `
      SELECT Track.Name as SongName, Artist.Name as ArtistName 
      FROM Album 
      LEFT JOIN Artist ON Album.ArtistId = Artist.ArtistId 
      LEFT JOIN Track ON Track.AlbumId = Album.AlbumId 
      WHERE Artist.Name LIKE '%${artist}%';
    `;
    var result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "get_tracks_by_artist",
    description: "Get songs by an artist (or similar artists).",
    schema: z.object({
      artist: z.string().describe("The artist name"),
    }),
  }
);

var getSongsByGenre = tool(
  async ({ genre }: { genre: string }) => {
    // First get genre ID
    var genreQuery = `SELECT GenreId FROM Genre WHERE Name LIKE '%${genre}%'`;
    var genreResult = await db.run(genreQuery);
    
    if (!genreResult || genreResult.length === 0) {
      return `No songs found for the genre: ${genre}`;
    }
    
    var genreIds = genreResult.map((row: any) => row.GenreId).join(", ");
    
    var songsQuery = `
      SELECT Track.Name as SongName, Artist.Name as ArtistName
      FROM Track
      LEFT JOIN Album ON Track.AlbumId = Album.AlbumId
      LEFT JOIN Artist ON Album.ArtistId = Artist.ArtistId
      WHERE Track.GenreId IN (${genreIds})
      GROUP BY Artist.Name
      LIMIT 8;
    `;
    
    var songs = await db.run(songsQuery);
    return JSON.stringify(songs);
  },
  {
    name: "get_songs_by_genre",
    description: "Fetch songs from the database that match a specific genre.",
    schema: z.object({
      genre: z.string().describe("The genre of the songs to fetch"),
    }),
  }
);

var checkForSongs = tool(
  async ({ songTitle }: { songTitle: string }) => {
    var query = `SELECT * FROM Track WHERE Name LIKE '%${songTitle}%';`;
    var result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "check_for_songs",
    description: "Check if a song exists by its name.",
    schema: z.object({
      songTitle: z.string().describe("The song title to search for"),
    }),
  }
);

var musicTools = [getAlbumsByArtist, getTracksByArtist, getSongsByGenre, checkForSongs];
var llmWithMusicTools = model.bindTools(musicTools);

console.log("Music tools defined!");


Music tools defined!


In [7]:
// Workaround for tslab module resolution
var { ToolNode } = require("@langchain/langgraph/prebuilt");
import { SystemMessage, HumanMessage } from "langchain";

// Create tool node
var musicToolNode = new ToolNode(musicTools);

// Helper function to generate music assistant prompt
function generateMusicAssistantPrompt(memory: string = "None"): string {
  return `
<important_background>
You are a member of the assistant team, your role specifically is to focused on helping customers discover and learn about music in our digital catalog. 
If you are unable to find playlists, songs, or albums associated with an artist, it is okay. 
Just respond that the catalog does not have any playlists, songs, or albums associated with that artist.
You also have context on any saved user preferences, helping you to tailor your response. 
IMPORTANT: Your interaction with the customer is done through an automated system. You are not directly interacting with the customer, so avoid chitchat or follow up questions and focus PURELY on responding to the request with the necessary information. 
</important_background>

<core_responsibilities>
- Search and provide accurate information about songs, albums, artists, and playlists
- Offer relevant recommendations based on customer interests
- Handle music-related queries with attention to detail
- Help customers discover new music they might enjoy
- You are routed only when there are questions related to music catalog; ignore other questions. 
</core_responsibilities>

<guidelines>
1. Always perform thorough searches before concluding something is unavailable
2. If exact matches aren't found, try:
   - Checking for alternative spellings
   - Looking for similar artist names
   - Searching by partial matches
   - Checking different versions/remixes
3. When providing song lists:
   - Include the artist name with each song
   - Mention the album when relevant
   - Note if it's part of any playlists
   - Indicate if there are multiple versions
</guidelines>

Additional context is provided below: 

Prior saved user preferences: ${memory}

Message history is also attached.  
`;
}

// Music assistant node
async function musicAssistant(state: typeof StateAnnotation.State) {
  // Fetching long term memory
  let memory = "None";
  if (state.loadedMemory) {
    memory = state.loadedMemory;
  }

  // Instructions for our agent
  var musicAssistantPrompt = generateMusicAssistantPrompt(memory);

  // Invoke the model
  var response = await llmWithMusicTools.invoke([
    new SystemMessage(musicAssistantPrompt),
    ...state.messages,
  ]);

  // Update the state
  return { messages: [response] };
}

console.log("Nodes defined!");


Nodes defined!


In [8]:
import { AIMessage } from "langchain";

// Conditional edge that determines whether to continue or not
function shouldContinue(state: typeof StateAnnotation.State): "continue" | "end" {
  var messages = state.messages;
  var lastMessage = messages[messages.length - 1] as AIMessage;

  // If there is no function call, then we finish
  if (!lastMessage.tool_calls || lastMessage.tool_calls.length === 0) {
    return "end";
  }
  // Otherwise if there is, we continue
  return "continue";
}

console.log("Conditional edge defined!");


Conditional edge defined!


#### Compile Graph!

Now that we've defined our State and Nodes, let's put it all together and construct our react agent!


In [9]:
import { StateGraph, START, END } from "@langchain/langgraph";
import { InMemoryStore, MemorySaver } from "@langchain/langgraph";

// NOTE: Due to tslab's module compilation, variables don't always persist across cells
// Re-initialize memory stores here if they're undefined
if (typeof checkpointer === 'undefined') {
  var checkpointer = new MemorySaver();
  var inMemoryStore = new InMemoryStore();
  console.log("Re-initialized memory stores");
}

// Create the workflow
var musicWorkflow = new StateGraph(StateAnnotation)
  // Add nodes
  .addNode("music_assistant", musicAssistant)
  .addNode("music_tool_node", musicToolNode)
  // Add edges
  .addEdge(START, "music_assistant")
  .addConditionalEdges("music_assistant", shouldContinue, {
    continue: "music_tool_node",
    end: END,
  })
  .addEdge("music_tool_node", "music_assistant");

// Compile the graph
var musicCatalogSubagent = musicWorkflow.compile({
  checkpointer,
  store: inMemoryStore,
});

console.log("Music catalog subagent compiled!");


Re-initialized memory stores
Music catalog subagent compiled!


#### Testing

Let's see how it works!


In [10]:
import { v4 as uuidv4 } from "uuid";

var threadId = uuidv4();
var question = "I like the Rolling Stones. What songs do you recommend by them or by other artists that I might like?";
var config: any = { configurable: { thread_id: threadId } };

var result = await musicCatalogSubagent.invoke(
  { messages: [new HumanMessage(question)] },
  config
);

// Print all messages
for (const message of result.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
  if (message._getType() === "ai" && (message as AIMessage).tool_calls) {
    console.log("Tool calls:", (message as AIMessage).tool_calls);
  }
}


[human]: I like the Rolling Stones. What songs do you recommend by them or by other artists that I might like?
[ai]: 
Tool calls: [
  {
    name: 'get_tracks_by_artist',
    args: { artist: 'The Rolling Stones' },
    type: 'tool_call',
    id: 'call_LfdcpHk2BGYmT6HFoAiZriLd'
  }
]
[tool]: "[{\"SongName\":\"Time Is On My Side\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"Heart Of Stone\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"Play With Fire\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"Satisfaction\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"As Tears Go By\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"Get Off Of My Cloud\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"Mother's Little Helper\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"19th Nervous Breakdown\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"Paint It Black\",\"ArtistName\":\"The Rolling Stones\"},{\"SongName\":\"Under My Thumb\",\"A

### 1.2. Building ReAct Agent using LangChain's `createAgent()`

LangChain offers a powerful ReAct agent architecture out of the box, allowing us to quickly create and iterate on applications that leverage this widespread design. More information of this pre-built architecture can be found [here](https://docs.langchain.com/oss/javascript/releases/langchain-v1#createagent)

In the last workflow, we have seen how we can build a ReAct agent from scratch. Now, we will show how we can leverage the LangChain pre-built ReAct agent to achieve similar results. 

![react_2](../../images/invoice_subagent.png)

Our **invoice info subagent** is responsible for all customer queries related to the invoices.


#### Defining tools and prompt

Similarly, let's first define a set of tools and our agent prompt below. 

**Note on customer ID**: The `customerId` is passed as a **regular parameter** to each tool. The LLM will extract and pass the customer ID based on the user's message or system context. This is the simplest approach for TypeScript LangChain v1.0.1, as state injection features (`ToolRuntime`, `InjectedState`) are not yet available in this version.

In [27]:
import { tool } from "langchain";
import { z } from "zod";

// Invoice tools with customerId as a regular parameter
// The LLM will pass the customer ID based on the system prompt context
var getInvoicesByCustomerSortedByDate = tool(
  async ({ customerId }: { customerId: number }) => {
    var query = `SELECT * FROM Invoice WHERE CustomerId = ${customerId} ORDER BY InvoiceDate DESC;`;
    var result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "get_invoices_by_customer_sorted_by_date",
    description: "Look up all invoices for a customer using their customer ID. The invoices are sorted in descending order by invoice date.",
    schema: z.object({
      customerId: z.number().describe("The customer ID"),
    }),
  }
);

var getInvoicesSortedByUnitPrice = tool(
  async ({ customerId }: { customerId: number }) => {
    var query = `
      SELECT Invoice.*, InvoiceLine.UnitPrice
      FROM Invoice
      JOIN InvoiceLine ON Invoice.InvoiceId = InvoiceLine.InvoiceId
      WHERE Invoice.CustomerId = ${customerId}
      ORDER BY InvoiceLine.UnitPrice DESC;
    `;
    var result = await db.run(query);
    return JSON.stringify(result);
  },
  {
    name: "get_invoices_sorted_by_unit_price",
    description: "Use this tool when the customer wants to know the details of one of their invoices based on the unit price/cost. This tool looks up all invoices for a customer and sorts by unit price.",
    schema: z.object({
      customerId: z.number().describe("The customer ID"),
    }),
  }
);

var getEmployeeByInvoiceAndCustomer = tool(
  async ({ invoiceId, customerId }: { invoiceId: number; customerId: number }) => {
    var query = `
      SELECT Employee.FirstName, Employee.Title, Employee.Email
      FROM Employee
      JOIN Customer ON Customer.SupportRepId = Employee.EmployeeId
      JOIN Invoice ON Invoice.CustomerId = Customer.CustomerId
      WHERE Invoice.InvoiceId = ${invoiceId} AND Invoice.CustomerId = ${customerId};
    `;
    var result = await db.run(query);
    
    if (!result || result.length === 0) {
      return `No employee found for invoice ID ${invoiceId} and customer identifier ${customerId}.`;
    }
    return JSON.stringify(result);
  },
  {
    name: "get_employee_by_invoice_and_customer",
    description: "This tool will take in an invoice ID and customer ID and return the employee information associated with the invoice.",
    schema: z.object({
      invoiceId: z.number().describe("The ID of the specific invoice"),
      customerId: z.number().describe("The customer ID"),
    }),
  }
);

var invoiceTools = [
  getInvoicesByCustomerSortedByDate,
  getInvoicesSortedByUnitPrice,
  getEmployeeByInvoiceAndCustomer,
];

console.log("Invoice tools defined!");


Invoice tools defined!


In [28]:
var invoiceSubagentPrompt = `
<important_background>
You are a subagent among a team of assistants. You are specialized for retrieving and processing invoice information. 
Invoices contain information such as song purchases and billing history. Only respond to questions if they relate in some way to billing, invoices, or purchases.  
If you are unable to retrieve the invoice information, respond that you are unable to retrieve the information.
IMPORTANT: Your interaction with the customer is done through an automated system. You are not directly interacting with the customer, so avoid chitchat or follow up questions and focus PURELY on responding to the request with the necessary information. 
</important_background>
 
<tools>
You have access to three tools. These tools enable you to retrieve and process invoice information from the database. Here are the tools:
- get_invoices_by_customer_sorted_by_date: Retrieves all invoices for a customer (requires customerId parameter)
- get_invoices_sorted_by_unit_price: Retrieves all invoices for a customer sorted by unit price (requires customerId parameter)
- get_employee_by_invoice_and_customer: Retrieves employee information for an invoice (requires invoiceId and customerId parameters)

IMPORTANT: All tools require a customerId parameter. You will receive the customer ID from the user's context or state. Pay attention to the customer ID and always pass it when calling these tools.
</tools>

<core_responsibilities>
- Retrieve and process invoice information from the database
- Provide detailed information about invoices, including customer details, invoice dates, total amounts, employees associated with the invoice, etc. when the customer asks for it.
- Always maintain a professional, friendly, and patient demeanor in your responses.
</core_responsibilities>

You may have additional context that you should use to help answer the customer's query. It will be provided to you below:
`

#### Using the LangChain out-of-the-box agents
Now, let's put them together by using the pre-built ReAct agent thats LangChain provide out-of-the-box!

In [29]:
import { createAgent } from "langchain";

var checkpointer = new MemorySaver();
var inMemoryStore = new InMemoryStore();
var model = await initChatModel("openai:o3-mini");

// Define the subagent using LangChain v1's createAgent
var invoiceInformationSubagent = createAgent({
  model,
  tools: invoiceTools,
  systemPrompt: invoiceSubagentPrompt,
  stateSchema: StateAnnotation,
  checkpointer,
  store: inMemoryStore,
});

console.log("Invoice information subagent created!");


Invoice information subagent created!


#### Testing!

Let's try our new agent out!


In [30]:
var threadId2 = uuidv4();
var question2 = "What was my most recent invoice, and who was the employee that helped me with it? My customer ID is 1.";
var config2: any = { configurable: { thread_id: threadId2 } };

var result2 = await invoiceInformationSubagent.invoke(
  { 
    messages: [new HumanMessage(question2)]
  } as any,
  config2
);

// Print all messages
for (const message of result2.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
  if (message._getType() === "ai" && (message as AIMessage).tool_calls) {
    console.log("Tool calls:", (message as AIMessage).tool_calls);
  }
}


[human]: What was my most recent invoice, and who was the employee that helped me with it? My customer ID is 1.
[ai]: 
Tool calls: [
  {
    name: 'get_invoices_by_customer_sorted_by_date',
    args: { customerId: 1 },
    type: 'tool_call',
    id: 'call_BVwmKIt6XCwrDv7D0TjRF0tJ'
  }
]
[tool]: "[{\"InvoiceId\":382,\"CustomerId\":1,\"InvoiceDate\":\"2025-08-07 00:00:00\",\"BillingAddress\":\"Av. Brigadeiro Faria Lima, 2170\",\"BillingCity\":\"São José dos Campos\",\"BillingState\":\"SP\",\"BillingCountry\":\"Brazil\",\"BillingPostalCode\":\"12227-000\",\"Total\":8.91},{\"InvoiceId\":327,\"CustomerId\":1,\"InvoiceDate\":\"2024-12-07 00:00:00\",\"BillingAddress\":\"Av. Brigadeiro Faria Lima, 2170\",\"BillingCity\":\"São José dos Campos\",\"BillingState\":\"SP\",\"BillingCountry\":\"Brazil\",\"BillingPostalCode\":\"12227-000\",\"Total\":13.86},{\"InvoiceId\":316,\"CustomerId\":1,\"InvoiceDate\":\"2024-10-27 00:00:00\",\"BillingAddress\":\"Av. Brigadeiro Faria Lima, 2170\",\"BillingCity\":

## Part 2: Building A Multi-Agent Architecture
Now that we have two sub-agents that have different capabilities, how do we make sure customer tasks are appropriately routed between them? 

This is where the supervisor oversees the workflow, invoking appropriate subagents for relevant inquiries.

A **multi-agent architecture** offers several key benefits:
- Specialization & Modularity – Each sub-agent is optimized for a specific task, improving system accuracy 
- Flexibility – Agents can be quickly added, removed, or modified without affecting the entire system

![supervisor](../../images/supervisor.png)

### Part 2.1. Building The Supervisor Agent

LangChain's **createAgent** abstraction is designed to be easily extended to accommodate multi-agent architectures. We can either call an entire sub-agent as a tool, or call a tool that hands-off control to a sub-agent. 

For this workshop, we will choose to call our invoice and music catalog subagents as tools.

#### Part 2.1.1 Writing the supervisor's prompt


In [47]:
// Supervisor prompt with instructions to extract customer ID from context
var supervisorPrompt = `
<background>
You are an expert customer support assistant for a digital music store. You can handle music catalog or invoice related question regarding past purchases, song or album availabilities. 
You are dedicated to providing exceptional service and ensuring customer queries are answered thoroughly, and have a team of subagents that you can use to help answer queries from customers. 
Your primary role is to delegate tasks to this multi-agent team in order to answer queries from customers. 
</background>

<important_instructions>
Always respond to the customer through summarizing the findings of the individual responses from subagents. 
If a question is unrelated to music or invoice, politely remind the customer regarding your scope of work. Do not answer unrelated answers.
Based on the existing steps that have been taken in the messages, your role is to call the appropriate subagent based on the users query.
</important_instructions>

<tools>
You have 2 tools available to delegate to the subagents on your team:
1. music_catalog_information_subagent: Call this tool to delegate to the music subagent. The music agent has access to user's saved music preferences. It can also retrieve information about the digital music store's music 
catalog (albums, tracks, songs, etc.) from the database. 
2. invoice_information_subagent: Call this tool to delegate to the invoice subagent. This subagent is able to retrieve information about a customer's past purchases or invoices 
from the database. 
</tools>
`;


#### Part 2.1.2 Building the supervisor's tools


In [48]:
// Create supervisor tools that delegate to subagents
var callInvoiceInformationSubagent = tool(
  async ({ query, customerId }: { query: string; customerId: number }) => {
    // Pass both the query and customerId to the invoice subagent
    var result = await invoiceInformationSubagent.invoke({
      messages: [new HumanMessage(`Customer ID: ${customerId}. ${query}`)],
    } as any);
    var subagentResponse = result.messages[result.messages.length - 1].content;
    return subagentResponse;
  },
  {
    name: "invoice_information_subagent",
    description: "An agent that can assist with all invoice-related queries. It can retrieve information about a customer's past purchases or invoices.",
    schema: z.object({
      query: z.string().describe("The query to send to the invoice subagent"),
      customerId: z.number().describe("The customer ID (get this from the user's context)"),
    }),
  }
);

var callMusicCatalogSubagent = tool(
  async ({ query }: { query: string }) => {
    var result = await musicCatalogSubagent.invoke({
      messages: [new HumanMessage(query)],
    } as any);
    var subagentResponse = result.messages[result.messages.length - 1].content;
    return subagentResponse;
  },
  {
    name: "music_catalog_subagent",
    description: "An agent that can assist with all music-related queries. This agent has access to user's saved music preferences. It can also retrieve information about the digital music store's music catalog (albums, tracks, songs, etc.) from the database.",
    schema: z.object({
      query: z.string().describe("The query to send to the music catalog subagent"),
    }),
  }
);

console.log("Supervisor tools defined!");


Supervisor tools defined!


#### Part 2.1.3 Putting it all together


In [49]:
// Create the supervisor agent
var model = await initChatModel("openai:o3-mini");
var checkpointer = new MemorySaver();
var inMemoryStore = new InMemoryStore();

var supervisor = createAgent({
  model,
  tools: [callInvoiceInformationSubagent, callMusicCatalogSubagent],
  systemPrompt: supervisorPrompt,
  stateSchema: StateAnnotation,
  checkpointer,
  store: inMemoryStore,
});

console.log("Supervisor agent created!");


Supervisor agent created!


Let's test it out!


In [50]:
var threadId3 = uuidv4();
var question3 = "how much was my most recent purchase and what albums do you have by the rolling stones? My customer ID is 1.";
var config3: any = { configurable: { thread_id: threadId3 } };

var result3 = await supervisor.invoke(
  { 
    messages: [new HumanMessage(question3)],
    customerId: 1
  } as any,
  config3
);

// Print all messages
for (const message of result3.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
  if (message._getType() === "ai" && (message as AIMessage).tool_calls) {
    console.log("Tool calls:", (message as AIMessage).tool_calls);
  }
}


[human]: how much was my most recent purchase and what albums do you have by the rolling stones? My customer ID is 1.
[ai]: 
Tool calls: [
  {
    name: 'invoice_information_subagent',
    args: {
      query: 'Retrieve the amount of the most recent purchase for the customer.',
      customerId: 1
    },
    type: 'tool_call',
    id: 'call_2RnevEutLVxBchPdcmFoIqZr'
  }
]
[tool]: The amount of the most recent purchase is 8.91.
[ai]: 
Tool calls: [
  {
    name: 'music_catalog_subagent',
    args: {
      query: 'Retrieve album titles by the Rolling Stones from our music catalog'
    },
    type: 'tool_call',
    id: 'call_bZz78eSGPTtPtcoCw9cIymyt'
  }
]
[tool]: The Rolling Stones album titles in our music catalog include: 
• Hot Rocks, 1964-1971 (Disc 1)
• No Security
• Voodoo Lounge
[ai]: Your most recent purchase was for $8.91. In addition, the Rolling Stones albums available in our catalog are:
• Hot Rocks, 1964-1971 (Disc 1)
• No Security
• Voodoo Lounge

Let me know if you need an

## Part 3: Adding customer verification through human-in-the-loop
We currently invoke our graph with a customer ID as the customer identifier, but realistically, we may not always have access to the customer identity. To solve this, we want to **first verify the customer information** before executing their inquiry with our supervisor agent. 

In this step, we will be showing a simple implementation of such a node, using **human-in-the-loop** to prompt the customer to provide their account information. 

![customer-input](../../images/human_input.png)


In this step, we will write two nodes: 
- **verify_info** node that verifies account information 
- **human_input** node that prompts user to provide additional information 

ChatModels support attaching a structured data schema to adhere response to. This is useful in scenarios like extracting information or categorizing.


In [64]:
// Schema for parsing user-provided account information
var UserInputSchema = z.object({
  identifier: z.string().describe("Identifier, which can be a customer ID, email, or phone number."),
});

var structuredLlm = model.withStructuredOutput(UserInputSchema);

var structuredSystemPrompt = `You are a customer service representative responsible for extracting customer identifier.
Only extract the customer's account information from the message history. 
If they haven't provided the information yet, return an empty string for the identifier`;

console.log("Structured extraction setup complete!");


Structured extraction setup complete!


In [89]:
// Helper function to look up customer ID from various identifiers
async function getCustomerIdFromIdentifier(identifier: string): Promise<number | null> {
  // Direct customer ID (numeric)
  if (/^\d+$/.test(identifier)) {
    return parseInt(identifier);
  }
  
  // Phone number lookup
  if (identifier.startsWith("+")) {
    // Normalize by removing spaces and parentheses for flexible matching
    var normalizedInput = identifier.replace(/[\s\(\)]/g, '');
    
    // Try exact match first
    var query = `SELECT CustomerId FROM Customer WHERE Phone = '${identifier}';`;
    var rawResult = await db.run(query);
    var result = typeof rawResult === 'string' ? JSON.parse(rawResult) : rawResult;
    
    if (result && result.length > 0) {
      return result[0].CustomerId;
    }
    
    // Try normalized match if exact match fails
    var queryAll = `SELECT CustomerId, Phone FROM Customer WHERE Phone LIKE '+%';`;
    var rawAllPhones = await db.run(queryAll);
    var allPhones = typeof rawAllPhones === 'string' ? JSON.parse(rawAllPhones) : rawAllPhones;
    
    for (const row of allPhones) {
      if (row.Phone) {
        var normalizedDb = row.Phone.replace(/[\s\(\)]/g, '');
        if (normalizedDb === normalizedInput) {
          return row.CustomerId;
        }
      }
    }
  }
  
  // Email lookup
  if (identifier.includes("@")) {
    var query = `SELECT CustomerId FROM Customer WHERE Email = '${identifier}';`;
    var rawResult = await db.run(query);
    var result = typeof rawResult === 'string' ? JSON.parse(rawResult) : rawResult;
    
    if (result && result.length > 0) {
      return result[0].CustomerId;
    }
  }
  
  return null;
}

console.log("Customer ID lookup function defined!");


Customer ID lookup function defined!


In [90]:
// Verify info node
async function verifyInfo(state: typeof StateAnnotation.State) {
  if (state.customerId === undefined) {
    var systemInstructions = `
You are a music store agent, where you are trying to verify the customer identity as the first step of the customer support process. 
You cannot support them until their account is verified. 
In order to verify their identity, one of their customer ID, email, or phone number needs to be provided.
If the customer has not provided their identifier, please ask them for it.
If they have provided the identifier but cannot be found, please ask them to revise it.

IMPORTANT: Do NOT ask any questions about their request, or make any attempt at addressing their request until their identity is verified. It is CRITICAL that you only ask about their identity for security purposes.
`;

    var userInput = state.messages[state.messages.length - 1];
    
    // Parse for customer ID
    var parsedInfo = await structuredLlm.invoke([
      new SystemMessage(structuredSystemPrompt),
      userInput,
    ]);
    
    // Extract details
    var identifier = parsedInfo.identifier;
    
    let customerId: number | null = null;
    // Attempt to find the customer ID
    if (identifier) {
      customerId = await getCustomerIdFromIdentifier(identifier);
    }
    
    if (customerId !== null) {
      var intentMessage = new AIMessage(
        `Thank you for providing your information! I was able to verify your account with customer id ${customerId}.`
      );
      return {
        customerId: customerId,
        messages: [intentMessage],
      };
    } else {
      var response = await model.invoke([
        new SystemMessage(systemInstructions),
        ...state.messages,
      ]);
      return { messages: [response] };
    }
  } else {
    // Customer ID already exists, pass through
    return {};
  }
}

console.log("Verify info node defined!");


Verify info node defined!


Now, let's create our human_input node. We will be prompting the user input through the `interrupt` function from LangGraph v1.


In [91]:
import { interrupt } from "@langchain/langgraph";

// Human input node
function humanInput(state: typeof StateAnnotation.State) {
  var userInput = interrupt("Please provide input.");
  return { messages: [new HumanMessage(userInput as string)] };
}

console.log("Human input node defined!");


Human input node defined!


Let's put this together!


In [92]:
const checkpointer = new MemorySaver();
const inMemoryStore = new InMemoryStore();

// Wrapper function for supervisor agent to use as a node
async function supervisorNode(state: typeof StateAnnotation.State) {
  var result = await supervisor.invoke(state as any);
  return {
    messages: result.messages,
  };
}

// Conditional edge
function shouldInterrupt(state: typeof StateAnnotation.State): "continue" | "interrupt" {
  if (state.customerId !== undefined) {
    return "continue";
  } else {
    return "interrupt";
  }
}

// Build the graph with human-in-the-loop
var multiAgentVerify = new StateGraph(StateAnnotation, {
  input: InputStateAnnotation,
})
  .addNode("verify_info", verifyInfo)
  .addNode("human_input", humanInput)
  .addNode("supervisor", supervisorNode)  // Use wrapper function
  .addEdge(START, "verify_info")
  .addConditionalEdges("verify_info", shouldInterrupt, {
    continue: "supervisor",
    interrupt: "human_input",
  })
  .addEdge("human_input", "verify_info")
  .addEdge("supervisor", END);

var multiAgentVerifyGraph = multiAgentVerify.compile({
  checkpointer,
  store: inMemoryStore,
});

console.log("Multi-agent with verification graph compiled!");


Multi-agent with verification graph compiled!


Let's test it out!


In [93]:
var threadId4 = uuidv4();
var question4 = "How much was my most recent purchase?";
var config4: any = { configurable: { thread_id: threadId4 } };

var result4 = await multiAgentVerifyGraph.invoke(
  { messages: [new HumanMessage(question4)] },
  config4
);

// Print all messages
for (const message of result4.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
}


[human]: How much was my most recent purchase?
[ai]: For your security, I need to verify your account. Could you please provide your customer ID, email, or phone number?


Resume from interrupt using Command:


In [94]:
import { Command } from "@langchain/langgraph";

// Resume from interrupt
var question5 = "My phone number is +55 (12) 3923-5555.";
var result5 = await multiAgentVerifyGraph.invoke(
  new Command({ resume: question5 }),
  config4
);

// Print all messages
for (const message of result5.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
}


[human]: How much was my most recent purchase?
[ai]: For your security, I need to verify your account. Could you please provide your customer ID, email, or phone number?
[human]: My phone number is +55 (12) 3923-5555.
[ai]: Thank you for providing your information! I was able to verify your account with customer id 1.
[ai]: 
[tool]: Your most recent purchase was for a total of $8.91.
[ai]: The invoice subagent confirmed that your most recent purchase was for a total of $8.91.


Now, if I ask a follow-up question in the same thread, our agent state stores our customer_id, not needing to verify again.


In [95]:
var question6 = "What albums do you have by U2?";
var result6 = await multiAgentVerifyGraph.invoke(
  { messages: [new HumanMessage(question6)] },
  config4
);

// Print all messages
for (const message of result6.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
}


[human]: How much was my most recent purchase?
[ai]: For your security, I need to verify your account. Could you please provide your customer ID, email, or phone number?
[human]: My phone number is +55 (12) 3923-5555.
[ai]: Thank you for providing your information! I was able to verify your account with customer id 1.
[ai]: 
[tool]: Your most recent purchase was for a total of $8.91.
[ai]: The invoice subagent confirmed that your most recent purchase was for a total of $8.91.
[human]: What albums do you have by U2?
[ai]: 
[tool]: Here are the albums by U2 available in our catalog:
• Achtung Baby
• All That You Can't Leave Behind
• B-Sides 1980-1990
• How To Dismantle An Atomic Bomb
• Pop
• Rattle And Hum
• The Best Of 1980-1990
• War
• Zooropa
• Instant Karma: The Amnesty International Campaign to Save Darfur
[ai]: The catalog subagent has provided the following list of U2 albums available in our store:

• Achtung Baby  
• All That You Can't Leave Behind  
• B-Sides 1980-1990  
• How T

## Part 4: Adding Long-Term Memory
Now that we have created an agent workflow that includes verification and execution, let's take it a step further. 

**Long term memory** lets you store and recall information between conversations. We have already initialized a long term memory store. 

![memory](../../images/memory.png)

In this step, we will add 2 nodes: 
- **load_memory** node that loads from the long term memory store
- **create_memory** node that saves any music interests that the customer has shared about themselves


In [100]:
// User profile structure for creating memory
var UserProfileSchema = z.object({
  customerId: z.string().describe("The customer ID of the customer"),
  musicPreferences: z.array(z.string()).describe("The music preferences of the customer"),
});

// Helper function to format user memory
function formatUserMemory(userData: any): string {
  var profile = userData.memory;
  let result = "";
  if (profile && profile.musicPreferences && profile.musicPreferences.length > 0) {
    result += `Music Preferences: ${profile.musicPreferences.join(", ")}`;
  }
  return result.trim();
}

// Load memory node
async function loadMemory(state: typeof StateAnnotation.State) {
  var userId = state.customerId?.toString();
  if (!userId) {
    return { loadedMemory: "" };
  }
  
  var namespace = ["memory_profile", userId];
  var existingMemory = await inMemoryStore.get(namespace, "user_memory");
  let formattedMemory = "";
  
  if (existingMemory && existingMemory.value) {
    formattedMemory = formatUserMemory(existingMemory.value);
  }
  
  return { loadedMemory: formattedMemory };
}

console.log("Load memory node defined!");


Load memory node defined!


In [101]:
var createMemoryPrompt = `You are an expert analyst that is observing a conversation that has taken place between a customer and a customer support assistant. The customer support assistant works for a digital music store, and has utilized a multi-agent team to answer the customer's request. 
You are tasked with analyzing the conversation that has taken place between the customer and the customer support assistant, and updating the memory profile associated with the customer. 
You specifically care about saving any music interest the customer has shared about themselves, particularly their music preferences to their memory profile.

<core_instructions>
1. The memory profile may be empty. If it's empty, you should ALWAYS create a new memory profile for the customer.
2. You should identify any music interest the customer during the conversation and add it to the memory profile **IF** it is not already present.
3. For each key in the memory profile, if there is no new information, do NOT update the value - keep the existing value unchanged.
4. ONLY update the values in the memory profile if there is new information.
</core_instructions>

<expected_format>
The customer's memory profile should have the following fields:
- customerId: the customer ID of the customer
- musicPreferences: the music preferences of the customer

IMPORTANT: ENSURE your response is an object with these fields.
</expected_format>

<important_context>
**IMPORTANT CONTEXT BELOW**
To help you with this task, I have attached the conversation that has taken place between the customer and the customer support assistant below, as well as the existing memory profile associated with the customer that you should either update or create. 

The conversation between the customer and the customer support assistant that you should analyze is as follows:
{conversation}

The existing memory profile associated with the customer that you should either update or create based on the conversation is as follows:
{memory_profile}

</important_context>

Reminder: Take a deep breath and think carefully before responding.
`;

// Create memory node
async function createMemory(state: typeof StateAnnotation.State) {
  var userId = state.customerId?.toString();
  if (!userId) {
    return {};
  }
  
  var namespace = ["memory_profile", userId];
  var formattedMemory = state.loadedMemory || "";
  
  var formattedSystemMessage = new SystemMessage(
    createMemoryPrompt
      .replace("{conversation}", JSON.stringify(state.messages))
      .replace("{memory_profile}", formattedMemory)
  );
  
  var updatedMemory = await model.withStructuredOutput(UserProfileSchema).invoke([formattedSystemMessage]);
  
  var key = "user_memory";
  await inMemoryStore.put(namespace, key, { memory: updatedMemory });
  
  return {};
}

console.log("Create memory node defined!");


Create memory node defined!


In [102]:
const checkpointer = new MemorySaver();
const inMemoryStore = new InMemoryStore();

// Wrapper function for supervisor agent to use as a node
async function supervisorNode(state: typeof StateAnnotation.State) {
  var result = await supervisor.invoke(state as any);
  return {
    messages: result.messages,
  };
}

// Build the final multi-agent graph with memory
var multiAgentFinal = new StateGraph(StateAnnotation, {
  input: InputStateAnnotation,
})
  .addNode("verify_info", verifyInfo)
  .addNode("human_input", humanInput)
  .addNode("load_memory", loadMemory)
  .addNode("supervisor", supervisorNode)
  .addNode("create_memory", createMemory)
  .addEdge(START, "verify_info")
  .addConditionalEdges("verify_info", shouldInterrupt, {
    continue: "load_memory",
    interrupt: "human_input",
  })
  .addEdge("human_input", "verify_info")
  .addEdge("load_memory", "supervisor")
  .addEdge("supervisor", "create_memory")
  .addEdge("create_memory", END);

var multiAgentFinalGraph = multiAgentFinal.compile({
  checkpointer,
  store: inMemoryStore,
});

console.log("Final multi-agent graph with memory compiled!");


Final multi-agent graph with memory compiled!


In [103]:
var threadId7 = uuidv4();
var question7 = "My phone number is +55 (12) 3923-5555. How much was my most recent purchase? What albums do you have by the Rolling Stones?";
var config7: any = { configurable: { thread_id: threadId7 } };

var result7 = await multiAgentFinalGraph.invoke(
  { messages: [new HumanMessage(question7)] },
  config7
);

// Print all messages
for (const message of result7.messages) {
  console.log(`[${message._getType()}]: ${message.content}`);
}


[human]: My phone number is +55 (12) 3923-5555. How much was my most recent purchase? What albums do you have by the Rolling Stones?
[ai]: Thank you for providing your information! I was able to verify your account with customer id 1.
[ai]: 
[tool]: The amount for the most recent purchase is 8.91.
[ai]: 
[tool]: The Rolling Stones albums in our catalog include:
• Hot Rocks, 1964-1971 (Disc 1)
• No Security
• Voodoo Lounge
[ai]: I’ve got your details. Your most recent purchase was for 8.91. Regarding the Rolling Stones, the albums available in our catalog are:

• Hot Rocks, 1964-1971 (Disc 1)  
• No Security  
• Voodoo Lounge

Let me know if you need any more information.


Let's take a look at the memory!


In [105]:
var userId = "1";
var namespace = ["memory_profile", userId];
var memory: any = await inMemoryStore.get(namespace, "user_memory");

if (memory && memory.value) {
  var savedMusicPreferences = memory.value.memory.musicPreferences;
  console.log("Saved music preferences:", savedMusicPreferences);
}


Saved music preferences: [ 'Rolling Stones' ]


## Evaluations

**Evaluations** are a quantitative way to measure performance of agents, which is important because LLMs don't always behave predictably — small changes in prompts, models, or inputs can significantly impact results. Evaluations provide a structured way to identify failures, compare changes across different versions of your application, and build more reliable AI applications.

Evaluations are made up of three components:

1. A **dataset test** inputs and expected outputs.
2. An **application or target function** that defines what you are evaluating, taking in inputs and returning the application output
3. **Evaluators** that score your target function's outputs.

![Evaluation](../../images/evals-conceptual.png) 

There are many ways you can evaluate an agent. Common types of agent evaluations include:

1. **Final Response**: Evaluate the agent's final response.
2. **Single step**: Evaluate any agent step in isolation (e.g., whether it selects the appropriate tool).
3. **Trajectory**: Evaluate whether the agent took the expected path (e.g., of tool calls) to arrive at the final answer.
4. **Multi-turn**: Simulate multi-turn conversations to evaluate agent performance over multiple interactions.

**Note**: Full evaluation examples would require integration with LangSmith and are beyond the scope of this notebook. However, the patterns shown in the Python version can be adapted to TypeScript with the appropriate LangSmith client setup.


## Summary

In this notebook, we've built a complete multi-agent customer support system using TypeScript and LangChain v1, including:

1. **ReAct Agents**: Built both from scratch and using `createAgent` (the v1 replacement for `createReactAgent`)
2. **Multi-Agent Architecture**: Created a supervisor that delegates to specialized subagents
3. **Human-in-the-Loop**: Implemented customer verification using the `interrupt` function
4. **Long-Term Memory**: Added memory storage and retrieval for personalized interactions
5. **Tool Injection**: Used `InjectedState` to pass state data to tools seamlessly

### Key LangChain v1 Migration Points:

- ✅ Used `createAgent` from `langchain` instead of deprecated `createReactAgent`
- ✅ Used `interrupt()` for human-in-the-loop interrupts
- ✅ Used `Command.resume()` to resume from interrupts
- ✅ Properly configured state with `Annotation` and reducers
- ✅ Used `InjectedState` for passing state to tools

For production deployments, consider:
- Adding comprehensive error handling
- Implementing proper logging and monitoring
- Setting up LangSmith for evaluation and tracing
- Adding rate limiting and authentication
- Testing edge cases and error scenarios
